# __Predicting Insurance Costs with Linear Regressions__

For this project we will be using [Medical Cost Data](https://www.kaggle.com/datasets/mirichoi0218/insurance?resource=download) from Kaggle. This data set contains iformation on medical insurance bills, which are associated to some demographic and personal characteristics from the person that received it.

Through this project, we will attempt to find a linear relationship between said characteristics and the __total medical cost__ of a customer's procedure. Success in this project would be important for a hospital to properly predict and anticipate its future revenue and procedures.

We can start by importing the relevant libraries and modules, and then importing our dataset from the link above:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

%matplotlib inline

insurance = pd.read_csv('insurance.csv')

In [2]:
insurance.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [3]:
insurance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [4]:
insurance.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


In [5]:
insurance['smoker'].value_counts(normalize=True)

no     0.795217
yes    0.204783
Name: smoker, dtype: float64

In [6]:
insurance['sex'].value_counts(normalize=True)

male      0.505232
female    0.494768
Name: sex, dtype: float64

In [7]:
insurance['region'].value_counts(normalize=True)

southeast    0.272048
southwest    0.242900
northwest    0.242900
northeast    0.242152
Name: region, dtype: float64

#### __Initial Observations__

So far, we have learned that there are 1338 separate transactions with no null values. There are three separate categorical columns: `sex` and `region` are well balanced, and according to the `smoker` column, about 20.5% of our customers are smokers.

We should also take a look at the distribution of our target variable, `charges`:

In [ ]:
insurance.hist('charges')

The charges in our dataset are __mostly less than $20,000__, but there are quite a few above that as well. Since this will make it hard to center our errors around 0, we could use log to transform `charges`.

In [ ]:
insurance["log_charges"] = np.log2(insurance["charges"])

insurance.hist("log_charges")

The log-transformed target variable is more centered, which will help with ensuring that our errors are unbiased.

#### __Exploring Correlations__

The next step we can take to explore useful predictors is finding the correlation between our target variable `charges` and the rest. Before we do that, we should convert `sex` and `smoker` into binary columns. Since we cannot do that with `region`, we will use a boxplot to explore each region's relationship with the cost of a procedure:

In [ ]:
# converting 'sex' and 'smoker' into binary columns
insurance['sex'] = insurance['sex'].apply(lambda x: 1 if x=='male' else 0)
insurance['smoker'] = insurance['smoker'].apply(lambda x: 1 if x=='yes' else 0)

In [ ]:
# display correlation between target variable and other columns
corr_charges = insurance.corr()['charges'].sort_values(ascending=False)
plt.bar(x=corr_charges.index, height = corr_charges, data=corr_charges)

In [ ]:
insurance.boxplot(column = ["log_charges"], by = "region")

In [ ]:
corr = insurance[['smoker', 'age', 'bmi', 'children', 'sex']].corr()
print(corr)

#### __Correlation Results__

We've found that:
- Being a smoker has a high correlation with high treatment costs, at 80% correlation
- Age is the next highest correlation, at 30% correlation
- `bmi`, `children` and `sex` all seem to have less than a 20% correlation
- There seem to be no observable differences in the distribution of `log_charges` within the different regions
- We also cannot find any particularly high correlation between the non-target variables


## __Dividing the Data__

Give what we've found, we will choose `smoker`, `age` and `bmi` as the variables to include in our model. With this in mind, we can split our data into a __training set__ and a __test set__. 

We need a training set to estimate the effect of these three variables on the `charges` column. We will also need a separate test set to check how accurate the predictions are. Since the training set is what sets said coefficients, the errors and accuracy we would get from predicting on the same set would not be realistic. This is why we need separate test data to validate our model with.

For this exercise, we will use 75% of rows as training data, and the other 25% for testing:


In [ ]:
# keeping relevant columns and splitting the data set into training and testing sets
X = insurance[['smoker', 'age', 'bmi']]
y = insurance[['log_charges']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 1)

## __Building the Model__

With our data split, we will use the __training set__ to build the model. We will use the LinearRegression() function we imported at the start of the exercise for this.

Once that is done, we will fit the model to our training data, explore the resulting model coefficients, calculate the $ R^2 $ and mean squared error (MSE), and then write some observations on the results:

In [ ]:
# build and fit the model to the training data
model = LinearRegression()
model.fit(X_train, y_train)

# get model prediction
y_pred = model.predict(X_train)

# get model coefficients
for coef_name, coef_result  in zip(X.columns, model.coef_[0]):
    print(f'The coefficient for {coef_name} is {coef_result:.2f}')

In [ ]:
# get R^2 and MSE
r2 = r2_score(y_train, y_pred)
mse = mean_squared_error(y_train, y_pred)

# get original scale MSE
exp_mse = np.exp(mse)

print(f'The coefficient of determination for our model is {r2:.2f} and the MSE on the original scale is {exp_mse:.2f}')

#### __Observations__:

- As expected, being a smoker seems to an effect multiple times higher in the final `charges` than the other two variables
- The coefficient of determination with the training set is 0.74. meaning that 74% of the variation in the log-scaled `charges` can be explained by the model. This is a good first step, but this is still the training set and therefore optimistic
- The MSE on the original scale in 1.58

## __Residual Diagnostics__

Before we continue, we should ensure that our model follows the assumptions of a linear model. This can be achieved by ensuring the residuals (difference between model results and predicted results) have a mean close to zero, and by checking that the variance is constant.

We can achieve this in our exercise by comparing  our residuals through the different charges:

In [ ]:

# Quick visual check of residuals
check = pd.DataFrame()
check["residuals"] = y_train - y_pred
check["fitted"] = y_pred

check.plot.scatter(x = "fitted", y = "residuals")

#### __Residual Observations__:

- While a lot of our observations fall close to zero, particularly for predicted results lower than 14, residuals are also spread around
- We expected an even band around zero, but residuals trend downwards as the predicted value increases

This does not necessarily make the model predictions unusable, but it puts into question the linear regression assumptions.

## Interpreting the model

Let's get our coefficients againL

In [ ]:
# get model coefficients
for coef_name, coef_result  in zip(X.columns, model.coef_[0]):
    print(f'The coefficient for {coef_name} is {coef_result:.2f}, or {((np.exp(coef_result)-1)*100):.1f}% on the regular scale.')

These tell us that:

- A year increase in the subject is associated with a 0.05 increase in the log charges, holding smoking status and bmi constant. About a 5% increase in the charges on the regular scale.
- A unit increase in the subject BMI is associated with a 0.02 increase in the log charges, holding smoking status and age constant. About a 1.5% increase in the charges on the regular scale.
- A smoker is associated with a 2.23 increase in the log charges, holding age and bmi constant. About a 830% increase in the charges on the regular scale.

Keep in mind that we are not concerned about if these changes are statistically significant, so we don't know if these associations are truly non-zero. Our primary goal is prediction.

## __Final Model Evaluation__

The training error in the training data will always be optimistic, since model is fitted on the same data. To judge the model's predictive ability, we will have to check how it performs with the test data instead, since these observations had no effect in the building of the model.

Let's use the test data to calculate results and MSE, and then compare these to the train data results:

In [ ]:
predictions = model.predict(X_test)
mse_test = mean_squared_error(y_test, predictions)
exp_mse_test = np.exp(mse_test)
r2_test = r2_score(y_test, predictions)

print(f'Using our model on the test data, we find that the MSE was {mse_test:.2f}, the exp mse was {exp_mse_test:.2f} and the coef. of determination was {r2_test:.2f}')

In [ ]:
# plotting actuals vs predictions

fig, axes = plt.subplots(1,2, figsize=(12,6))
axes[0].scatter(predictions, y_test, alpha=.5)
axes[0].plot([0,y_test.max()],[0,y_test.max()], lw=1, color='red', alpha=.5)
axes[0].set_ylabel("Actual")
axes[0].set_ylim([8, 16])
axes[0].set_xlabel("Predicted")
axes[0].set_xlim([10, 16])
axes[0].set_title("Actual vs Predicted Values")

residuals = y_test - predictions
axes[1].scatter(predictions, residuals)
axes[1].set_xlabel("Prediction")
axes[1].set_ylabel("Residual")
axes[1].set_title("Residual Plot")

fig.tight_layout()
plt.show()

## __Conclusions__

The test MSE on the original scale was about 1.55, while the training original scale MSE was about 1.58. In this case, the two errors match up pretty well, so we can conclude that the model is not overfit. The residuals suggest that the model is predicting much lower costs for subjects who were actually charged much higher. Therefore the model struggles with these higher costs. As a whole, the model predictions are too conservative.

We might improve the model by including more complex terms in the regression, such as interactions or quadratic terms.